# Function to eval total energies (for degeneracy); Initial runs

In [1]:
from virp import CIFSupercell, PermutativeFill
from virp.enumerate import EquivalentStructures
from chgnet.model import CHGNet
from pymatgen.core.structure import Structure
import warnings
from pathlib import Path
import pandas as pd

# Load CHGNET zeropoint
model = CHGNet.load()

def SVC(input_cif, supercell, fname, sample_size=400, chgnetmodel = model):
    """
    Given a disordered .cif file, create an output folder
    containing a number (sample_size) of virtual cells
    
    Args:
        input_cif (str): Path to .cif (disordered)
        supercell [int,int,int]: multiplicity of supercell
        sample_size (int): Number of virtual cells to generate (default is 400)
        
    Returns:
        void
    """

    # Suppress warnings in this block
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        # Init data
        data = []

        # Make output folder directory
        Path(fname).mkdir(exist_ok=True)  # `exist_ok=True` avoids errors if the directory exists.
        print(f"Directory created at: {fname}")

        # Make the supercell
        sc_file = Path(fname) / (str(Path(fname)) + "_supercell.cif")
        CIFSupercell(input_cif, sc_file, supercell)

        # Execution
        for i in range(sample_size):
            # Permutative fill only, no structure optimization
            pfill_file = Path(fname) / Path("_virtual_"+str(i)+".cif")
            PermutativeFill(sc_file, pfill_file, verbose = False)
            print(f"\rGenerating virtual cell #{i} ({i+1}/{sample_size})", end="", flush=True)

            # Predict total energy
            structure = Structure.from_file(pfill_file)
            prediction = model.predict_structure(structure)
            total_energy = prediction["e"]  # Energy in eV

            # data append
            data.append({
                "virtual-id": i,
                "Total Energy (eV)": total_energy,
            })

        # Create a DataFrame
        df = pd.DataFrame(data)

        # Save the DataFrame to a CSV file
        output_file = Path(fname) / "total_energies.csv"
        df.to_csv(output_file, index = False)
        print(f"Results saved to {output_file}")

        # Perform enumeration
        EquivalentStructures(output_file)
        
        with open(Path(fname) / "_JOBDONE", 'w') as file: pass # make an empty file signalling completion
        print("\nAll cells generated (see _JOBDONE file).")

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu


# Main Body of Code (serious runs)

In [3]:
brass_file = "197803-brass-Cu62Zn38.cif"

In [9]:
SVC(brass_file, [3,3,3], "333", sample_size=400, chgnetmodel = model) 

Directory created at: 333
Supercell created and saved as  333\333_supercell.cif
Generating virtual cell #399 (400/400)Results saved to 333\total_energies.csv
Number of redundant structures: 2/400 (0.50%)

All cells generated (see _JOBDONE file).


In [28]:
SVC(brass_file, [1,1,1], "111", sample_size=400, chgnetmodel = model) 

Directory created at: 111
Supercell created and saved as  111\111_supercell.cif
Disordered site name:  Zn1
- Number of elements in this site:  2
- Number of sites in supercell:  4
- Atoms and site assignment (float/rounded):  [('Zn', 1.52, 2), ('Cu', 2.48, 2)]
- No of filled sites:  4 / 4
Generating virtual cell #399 (400/400)Results saved to 111\total_energies.csv
Number of redundant structures: 398/400 (99.50%)

All cells generated (see _JOBDONE file).


In [5]:
SVC(brass_file, [1,1,2], "112", sample_size=400, chgnetmodel = model) 

Directory created at: 112
Supercell created and saved as  112\112_supercell.cif
Generating virtual cell #399 (400/400)Results saved to 112\total_energies.csv
Number of redundant structures: 392/400 (98.00%)

All cells generated (see _JOBDONE file).


In [7]:
SVC(brass_file, [1,2,2], "122", sample_size=400, chgnetmodel = model) 

Directory created at: 122
Supercell created and saved as  122\122_supercell.cif
Generating virtual cell #399 (400/400)Results saved to 122\total_energies.csv
Number of redundant structures: 277/400 (69.25%)

All cells generated (see _JOBDONE file).


In [6]:
SVC(brass_file, [1,2,3], "123", sample_size=400, chgnetmodel = model) 

Directory created at: 123
Supercell created and saved as  123\123_supercell.cif
Generating virtual cell #399 (400/400)Results saved to 123\total_energies.csv
Number of redundant structures: 14/400 (3.50%)

All cells generated (see _JOBDONE file).


In [7]:
SVC(brass_file, [1,1,3], "113", sample_size=400, chgnetmodel = model) 

Directory created at: 113
Supercell created and saved as  113\113_supercell.cif
Generating virtual cell #399 (400/400)Results saved to 113\total_energies.csv
Number of redundant structures: 352/400 (88.00%)

All cells generated (see _JOBDONE file).


In [8]:
SVC(brass_file, [3,2,2], "322", sample_size=400, chgnetmodel = model) 

Directory created at: 322
Supercell created and saved as  322\322_supercell.cif
Generating virtual cell #399 (400/400)Results saved to 322\total_energies.csv
Number of redundant structures: 4/400 (1.00%)

All cells generated (see _JOBDONE file).


In [9]:
SVC(brass_file, [3,3,2], "332", sample_size=400, chgnetmodel = model) 

Directory created at: 332
Supercell created and saved as  332\332_supercell.cif
Generating virtual cell #399 (400/400)Results saved to 332\total_energies.csv
Number of redundant structures: 0/400 (0.00%)

All cells generated (see _JOBDONE file).


# Test run

In [26]:
SVC("197803-brass-Cu62Zn38.cif", [2,2,2], "TEST", sample_size=3, chgnetmodel = model)

Directory created at: TEST
Supercell created and saved as  TEST\TEST_supercell.cif
Disordered site name:  Zn1
- Number of elements in this site:  2
- Number of sites in supercell:  32
- Atoms and site assignment (float/rounded):  [('Zn', 12.16, 12), ('Cu', 19.84, 20)]
- No of filled sites:  32 / 32
Generating virtual cell #2 (3/3)Results saved to TEST\total_energies.csv
Number of redundant structures: 0/3 (0.00%)

All cells generated (see _JOBDONE file).


# Test CHGNET, total energy only

In [4]:
from pymatgen.core import Structure
from chgnet.model import CHGNet

# Load CIF file
cif_file = "CoFe_virtual_2-TEST.cif"  # Change this to your file path
# cif_file = "197803-brass-Cu62Zn38.cif"  # disordered?? no work
structure = Structure.from_file(cif_file)

# Load pre-trained CHGNet model
model = CHGNet.load()

# Predict total energy
prediction = model.predict_structure(structure)
total_energy = prediction["e"]  # Energy in eV

print(f"Total Energy: {total_energy:.4f} eV")

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
Total Energy: -7.9596 eV
